In [88]:
import pandas as pd
import numpy as np
import time

In [56]:
graph = pd.read_csv(
    "data/opsahl-ucsocial/out.opsahl-ucsocial", 
    names=["from", "to", "weight", "timestamp"],
    sep=" |\t",
    skiprows=[0, 1]
)
print(graph.head())

   from  to  weight   timestamp
0     1   2       1  1082008561
1     3   4       1  1082123439
2     5   2       1  1082381991
3     6   7       1  1082407219
4     8   7       1  1082407356


/var/folders/wc/28hh_w_d5zn4rvtk1bw0_g700000gn/T/ipykernel_4794/3009724296.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  graph = pd.read_csv(


In [57]:
V = np.unique(graph["from"]._append(graph["to"]))
n = V.size
volume = graph["timestamp"].size
print(f"{n = }, {volume = }")

n = 1899, volume = 59835


In [100]:
graph_static = [set() for _ in range(n +1)]
for row_number, row in graph.iterrows():
    graph_static[row["from"]].add(row["to"])
    graph_static[row["to"]].add(row["from"])


In [155]:
E_count = 0
for i in range(len(graph_static)):
    E_count += len(graph_static[i])
E_count //= 2
print(E_count)

p = E_count * 2 / (n * (n - 1))
print(p)

13838
0.007678601848568738


In [169]:
V_to_visit = set(V)
connectivity_components = []
while(V_to_visit):
    V_visited = set()
    queue = []
    for u in V_to_visit:
        queue.append(u)
        break
    while queue:
        u = queue.pop()
        V_to_visit.discard(u)
        V_visited.add(u)
        u_adjacent_to_visit = graph_static[u].difference(V_visited)
        for v in u_adjacent_to_visit:
            if v not in queue:
                queue.append(v)
    connectivity_components.append(V_visited)

sizes = list(map(lambda x: len(x), connectivity_components))
max_val = max(sizes)
max_connectivity_component_index = sizes.index(max_val)
proportion = max_val / len(V)
print(f"{max_connectivity_component_index = }, {proportion = }")

       
    


max_connectivity_component_index = 0, proportion = 0.9968404423380727


In [170]:
# 1.1
print("|V| = %i, |E| = %i, p = %f, number of components = %i, max component proportion = %f" 
      % (n, E_count, p, len(connectivity_components), proportion))

|V| = 1899, |E| = 13838, p = 0.007679, number of components = 4, max component proportion = 0.996840


In [167]:
component = list(connectivity_components[max_connectivity_component_index])
eccentricities = dict()
for start in component:
    V_visited = set()
    queue = [(start, 0)]
    queued = set([start])
    max_depth = 0
    while queue:
        u, depth = queue.pop(0)
        max_depth = max(max_depth, depth)
        queued.discard(u)
        V_visited.add(u)
        u_adjacent_to_visit = graph_static[u].difference(V_visited)
        for v in u_adjacent_to_visit:
            if v not in queued:
                queue.append((v, depth + 1))
                queued.add(v)
    eccentricities[start] = max_depth

8


In [121]:
# start = time.time()
# a = [] * (10**6)
# for i in range(len(a)):
#     a[i] = set()
# end = time.time()
# print(end - start)

# start = time.time()
# a = [set() for i in range(10**6)]
# end = time.time()
# print(end - start)



In [168]:
diameter = max(eccentricities.values())
radius = min(eccentricities.values())

In [171]:
# 1.2
print("diameter = %i, raduis = %i" 
      % (diameter, radius))

diameter = 8, raduis = 4


In [ ]:
#1.3
print("Cl_average = %f" % (0))